In [168]:
import pandas as pd

MA1 = pd.DataFrame()
for chunk in pd.read_csv("../ssd_data/combined_by_diskmodel/combined_MA2_preprocessed.csv", chunksize=1000000):
    MA1 = pd.concat([MA1, chunk], ignore_index=True)
MA1

,disk_id,ds,n_5,r_5,r_9,r_12,n_170,n_171,r_171,n_172,...,r_194,r_197,r_199,n_233,r_241,r_242,n_175,r_175,n_232,label
0,100061,20190101,100.0,0.0,38952.0,12.0,100.0,100.0,0.0,100.0,...,22.0,0.0,0.0,81.0,6776719.0,330779.0,100.0,9.965843e+11,100.0,0
1,62439,20190101,100.0,0.0,38445.0,23.0,100.0,100.0,0.0,100.0,...,30.0,0.0,6.0,84.0,5017850.0,7962409.0,100.0,9.665438e+11,100.0,0
2,6243,20190101,100.0,0.0,30463.0,9.0,100.0,100.0,0.0,100.0,...,26.0,0.0,9.0,99.0,904637.0,1778683.0,100.0,7.819016e+11,100.0,0
3,62409,20190101,100.0,0.0,28414.0,57.0,100.0,100.0,0.0,100.0,...,26.0,0.0,0.0,96.0,2077567.0,28457.0,100.0,7.046157e+11,100.0,0
4,6231,20190101,100.0,0.0,30623.0,15.0,100.0,100.0,0.0,100.0,...,25.0,0.0,0.0,92.0,4791692.0,717317.0,100.0,7.860970e+11,100.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17217442,22645,20190630,100.0,0.0,33282.0,12.0,100.0,100.0,0.0,100.0,...,25.0,0.0,0.0,98.0,1123083.0,3844475.0,100.0,8.504329e+11,100.0,0
17217443,22643,20190630,100.0,0.0,33280.0,12.0,100.0,100.0,0.0,100.0,...,23.0,0.0,0.0,97.0,1134357.0,3845992.0,100.0,8.504278e+11,100.0,0
17217444,22476,20190630,100.0,0.0,33554.0,10.0,100.0,100.0,0.0,100.0,...,26.0,0.0,0.0,29.0,17961393.0,799979.0,100.0,8.590448e+11,100.0,0
17217445,22406,20190630,100.0,0.0,32906.0,22.0,100.0,100.0,0.0,100.0,...,30.0,0.0,61.0,81.0,5670231.0,3506879.0,100.0,8.294693e+11,100.0,0


In [138]:
#del(MA1['index'])

In [174]:
# null값이 있는 열을 제거하는 코드
def remove_null_columns(df):
    # null인 컬럼을 확인하고, 모든 값이 null인 컬럼을 찾습니다.
    null_columns = df.columns[df.isnull().all()]
    
    # 모든 값이 null인 컬럼을 DataFrame에서 제거합니다.
    df = df.drop(null_columns, axis=1)
    
    return df

In [175]:
# 하나의 unique한 값으로만 이루어진 컬럼을 제거하는 코드
def remove_single_value_columns(df):
    single_value_cols = []
    for col in df.columns:
        unique_values = df[col].nunique()
        if unique_values == 1:
            single_value_cols.append(col)

    df_filtered = df.drop(single_value_cols, axis=1)
    return df_filtered


MA1을 기준으로 다시 컬럼이 NULL값이 있는 열과 단일 변수로 이루어진 열을 제거한다.

In [176]:
MA1 = remove_null_columns(MA1)
MA1 = remove_single_value_columns(MA1)

In [177]:
MA1=MA1.fillna(-1)

디스크가 fail된 아이디를 가져와서 그 fail된 디스크아이디를 가진 모든 행을 따로 만들어서 전처리를 구현한다.

In [178]:
# 라벨이 1인 행을 필터링합니다.
selected_disk_model_1 = MA1[MA1['label'] == 1]

# 모델, disk_id, 그리고 날짜 순으로 정렬합니다.
selected_disk_model_1 = selected_disk_model_1.sort_values(by=['disk_id', 'ds'])

# disk_id와 model 열만을 선택하여 새로운 데이터 프레임에 저장합니다.
selected_disk_model_1 = selected_disk_model_1[['disk_id']].drop_duplicates().reset_index(drop=True)

# 결과 데이터 프레임을 출력합니다.
selected_disk_model_1.shape

(34, 1)

In [179]:
# disk_ids 데이터프레임에 있는 disk_id와 일치하는 행을 MA1 데이터프레임에서 불러옴
selected_label_1 = MA1[MA1['disk_id'].isin(selected_disk_model_1['disk_id'])]

# ds 컬럼에 따라 정렬
selected_label_1 = selected_label_1.sort_values(by=['disk_id','ds'])
# 결과 출력
selected_label_1

,disk_id,ds,n_5,r_5,r_9,r_12,n_170,n_171,r_171,n_172,...,r_194,r_197,r_199,n_233,r_241,r_242,n_175,r_175,n_232,label
56799,2321,20190101,100.0,0.0,34205.0,25.0,100.0,100.0,0.0,100.0,...,26.0,0.0,193.0,95.0,1529086.0,4339011.0,100.0,8.595186e+11,100.0,0
184144,2321,20190102,100.0,0.0,34228.0,25.0,100.0,100.0,0.0,100.0,...,25.0,0.0,193.0,95.0,1529086.0,4339011.0,100.0,8.596125e+11,100.0,0
245209,2321,20190103,100.0,0.0,34252.0,25.0,100.0,100.0,0.0,100.0,...,24.0,0.0,193.0,95.0,1529086.0,4339011.0,100.0,8.633409e+11,100.0,0
341133,2321,20190104,100.0,0.0,34276.0,25.0,100.0,100.0,0.0,100.0,...,24.0,0.0,193.0,95.0,1529086.0,4339011.0,100.0,8.634345e+11,100.0,0
504251,2321,20190105,100.0,0.0,34300.0,25.0,100.0,100.0,0.0,100.0,...,25.0,0.0,193.0,95.0,1529086.0,4339011.0,100.0,8.635284e+11,100.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12482590,113915,20190508,100.0,0.0,34181.0,12.0,100.0,100.0,0.0,100.0,...,25.0,0.0,0.0,1.0,26027256.0,2481040.0,100.0,8.723547e+11,100.0,0
12574906,113915,20190509,100.0,0.0,34205.0,12.0,100.0,100.0,0.0,100.0,...,25.0,0.0,0.0,1.0,26029403.0,2481040.0,100.0,8.724490e+11,100.0,0
12677963,113915,20190510,100.0,0.0,34229.0,12.0,100.0,100.0,0.0,100.0,...,27.0,0.0,0.0,1.0,26031931.0,2481040.0,100.0,8.761778e+11,100.0,0
12772120,113915,20190511,100.0,0.0,34253.0,12.0,100.0,100.0,0.0,100.0,...,27.0,0.0,0.0,1.0,26034528.0,2481040.0,100.0,8.762722e+11,100.0,0


In [180]:
selected_label_1['ds'] = pd.to_datetime(selected_label_1['ds'], format='%Y%m%d')

그리고 fail된 디스크만 있는 dataframe에서 fail이후에도 디스크가 돌아가는 것이 있어서 그 행들을 다 제거하는 과정을 거쳤다.

In [181]:
remove_indexes = []
for index, row in selected_label_1.iterrows():
    if row['label'] == 1:
        disk_id = row['disk_id']
        ds = row['ds']
        remove_indexes.extend(selected_label_1.index[(selected_label_1['disk_id'] == disk_id) & (selected_label_1['ds'] > ds)].tolist())

# 중복된 인덱스를 제거하여 제거할 행들을 구함
remove_indexes = list(set(remove_indexes))

# 제거할 행을 제외한 최종 DataFrame을 생성
del_fail_after_row = selected_label_1.drop(index=remove_indexes)

# 결과 출력
del_fail_after_row

,disk_id,ds,n_5,r_5,r_9,r_12,n_170,n_171,r_171,n_172,...,r_194,r_197,r_199,n_233,r_241,r_242,n_175,r_175,n_232,label
56799,2321,2019-01-01,100.0,0.0,34205.0,25.0,100.0,100.0,0.0,100.0,...,26.0,0.0,193.0,95.0,1529086.0,4339011.0,100.0,8.595186e+11,100.0,0
184144,2321,2019-01-02,100.0,0.0,34228.0,25.0,100.0,100.0,0.0,100.0,...,25.0,0.0,193.0,95.0,1529086.0,4339011.0,100.0,8.596125e+11,100.0,0
245209,2321,2019-01-03,100.0,0.0,34252.0,25.0,100.0,100.0,0.0,100.0,...,24.0,0.0,193.0,95.0,1529086.0,4339011.0,100.0,8.633409e+11,100.0,0
341133,2321,2019-01-04,100.0,0.0,34276.0,25.0,100.0,100.0,0.0,100.0,...,24.0,0.0,193.0,95.0,1529086.0,4339011.0,100.0,8.634345e+11,100.0,0
504251,2321,2019-01-05,100.0,0.0,34300.0,25.0,100.0,100.0,0.0,100.0,...,25.0,0.0,193.0,95.0,1529086.0,4339011.0,100.0,8.635284e+11,100.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8025468,113915,2019-03-21,100.0,0.0,33030.0,12.0,100.0,100.0,0.0,100.0,...,23.0,0.0,0.0,1.0,25890000.0,2481028.0,100.0,8.423889e+11,100.0,0
8086678,113915,2019-03-22,100.0,0.0,33054.0,12.0,100.0,100.0,0.0,100.0,...,26.0,0.0,0.0,1.0,25927385.0,2481029.0,100.0,8.461175e+11,100.0,0
8195276,113915,2019-03-23,100.0,0.0,33078.0,12.0,100.0,100.0,0.0,100.0,...,26.0,0.0,0.0,1.0,25930013.0,2481029.0,100.0,8.462118e+11,100.0,0
8278283,113915,2019-03-24,100.0,0.0,33102.0,12.0,100.0,100.0,0.0,100.0,...,26.0,0.0,0.0,1.0,25932642.0,2481029.0,100.0,8.463061e+11,100.0,0


In [182]:
selected_label_1[(selected_label_1['disk_id']==113915) & (selected_label_1['label']==1)]

,disk_id,ds,n_5,r_5,r_9,r_12,n_170,n_171,r_171,n_172,...,r_194,r_197,r_199,n_233,r_241,r_242,n_175,r_175,n_232,label
8369561,113915,2019-03-25,100.0,0.0,33126.0,12.0,100.0,100.0,0.0,100.0,...,24.0,0.0,0.0,1.0,25934001.0,2481029.0,100.0,8.464003e+11,100.0,1


그 뒤에 30일 이전에 대해서 라벨링을 한다.
이때 주의해야할 점이 많은 디스크들이 fail되기 이전에 30일전이 연속적으로 있는게 아니라 중간에 비어있는 날짜들이 있다.
그래서 날짜를 기준으로 한달전을 label을 바꾼다. 만약 30일 전이 25일로 이루어져있으면 25개가 1로 라벨링 되는것이다.

In [193]:
import pandas as pd

# 'ds' 컬럼을 datetime 형태로 변환
selected_label_1['ds'] = pd.to_datetime(selected_label_1['ds'], format='%Y%m%d')

# 모든 행에 대해 반복
for index, row in selected_label_1.iterrows():
    # label 값이 1인 경우
    if row['label'] == 1:
        # 해당 행의 disk_id
        current_disk_id = row['disk_id']
        # 해당 행의 날짜
        current_date = row['ds']
        # 30일 이전의 날짜 계산
        days_before = current_date - pd.Timedelta(days=30)
        
        # 같은 disk_id이면서, 날짜가 30일 이내인 행들을 찾아서 label 값을 1로 설정
        selected_label_1.loc[(selected_label_1['disk_id'] == current_disk_id) &
                             (selected_label_1['ds'] <= current_date) &
                             (selected_label_1['ds'] > days_before), 'label'] = 1

In [184]:
del_fail_after_row[del_fail_after_row['label']==1]

,disk_id,ds,n_5,r_5,r_9,r_12,n_170,n_171,r_171,n_172,...,r_194,r_197,r_199,n_233,r_241,r_242,n_175,r_175,n_232,label
56799,2321,2019-01-01,100.0,0.0,34205.0,25.0,100.0,100.0,0.0,100.0,...,26.0,0.0,193.0,95.0,1529086.0,4339011.0,100.0,8.595186e+11,100.0,1
184144,2321,2019-01-02,100.0,0.0,34228.0,25.0,100.0,100.0,0.0,100.0,...,25.0,0.0,193.0,95.0,1529086.0,4339011.0,100.0,8.596125e+11,100.0,1
245209,2321,2019-01-03,100.0,0.0,34252.0,25.0,100.0,100.0,0.0,100.0,...,24.0,0.0,193.0,95.0,1529086.0,4339011.0,100.0,8.633409e+11,100.0,1
341133,2321,2019-01-04,100.0,0.0,34276.0,25.0,100.0,100.0,0.0,100.0,...,24.0,0.0,193.0,95.0,1529086.0,4339011.0,100.0,8.634345e+11,100.0,1
504251,2321,2019-01-05,100.0,0.0,34300.0,25.0,100.0,100.0,0.0,100.0,...,25.0,0.0,193.0,95.0,1529086.0,4339011.0,100.0,8.635284e+11,100.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8025468,113915,2019-03-21,100.0,0.0,33030.0,12.0,100.0,100.0,0.0,100.0,...,23.0,0.0,0.0,1.0,25890000.0,2481028.0,100.0,8.423889e+11,100.0,1
8086678,113915,2019-03-22,100.0,0.0,33054.0,12.0,100.0,100.0,0.0,100.0,...,26.0,0.0,0.0,1.0,25927385.0,2481029.0,100.0,8.461175e+11,100.0,1
8195276,113915,2019-03-23,100.0,0.0,33078.0,12.0,100.0,100.0,0.0,100.0,...,26.0,0.0,0.0,1.0,25930013.0,2481029.0,100.0,8.462118e+11,100.0,1
8278283,113915,2019-03-24,100.0,0.0,33102.0,12.0,100.0,100.0,0.0,100.0,...,26.0,0.0,0.0,1.0,25932642.0,2481029.0,100.0,8.463061e+11,100.0,1


In [185]:
del_fail_after_row['label'].value_counts()

0    1189
1     824
Name: label, dtype: int64

In [186]:
del_fail_after_row[(selected_label_1['disk_id']==29444) & (selected_label_1['label']==1)]

C:\Users\choib\AppData\Local\Temp\ipykernel_46088\1860598359.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del_fail_after_row[(selected_label_1['disk_id']==29444) & (selected_label_1['label']==1)]


,disk_id,ds,n_5,r_5,r_9,r_12,n_170,n_171,r_171,n_172,...,r_194,r_197,r_199,n_233,r_241,r_242,n_175,r_175,n_232,label


In [153]:
selected_label_1[(selected_label_1['disk_id']==17978) & (selected_label_1['label']==1)]

,disk_id,ds,r_1,n_5,r_5,r_9,r_12,n_171,r_171,n_172,...,r_194,r_195,n_196,r_196,r_197,r_198,r_199,n_175,r_175,label
467264,17978,2019-01-17,4.294967e+09,94.0,0.0,47035.0,31.0,100.0,0.0,100.0,...,15.0,579652045.0,100.0,0.0,0.0,0.0,0.0,1.0,1.190354e+12,1


In [187]:
selected_disk_model_1

,disk_id
0,2321
1,3283
2,4653
3,7471
4,13576
5,20714
6,22659
7,35347
8,38765
9,42541


fail된 디스크가 275개라서 healthy한 디스크도 275개를 가져와서 병합하는 작업을 거친다.

In [188]:
import random

# 시드 값을 설정합니다.
random.seed(42)

# disk_id만 추출
disk_id_list = MA1['disk_id'].drop_duplicates()
selected_disk_model_0 = list(set(disk_id_list.values.squeeze()) - set(selected_disk_model_1.values.squeeze()))
selected_disk_model_0

# 275개를 랜덤하게 선택
selected_disk_model_0 = random.sample(selected_disk_model_0, 50)

In [189]:
# disk_ids 데이터프레임에 있는 disk_id와 일치하는 행을 MA1 데이터프레임에서 불러옴
selecetd_label_0= MA1[MA1['disk_id'].isin(selected_disk_model_0)]

# ds 컬럼에 따라 정렬
selecetd_label_0= selecetd_label_0.sort_values(by='ds')


# 'ds' 컬럼을 datetime 형태로 변환
selecetd_label_0['ds'] = pd.to_datetime(selecetd_label_0['ds'], format='%Y%m%d')

# 결과 출력
selecetd_label_0

,disk_id,ds,n_5,r_5,r_9,r_12,n_170,n_171,r_171,n_172,...,r_194,r_197,r_199,n_233,r_241,r_242,n_175,r_175,n_232,label
3716,106434,2019-01-01,100.0,0.0,31875.0,33.0,100.0,100.0,0.0,100.0,...,22.0,0.0,31.0,99.0,38970.0,4657321.0,100.0,8.161857e+11,100.0,0
59156,84725,2019-01-01,100.0,0.0,30337.0,15.0,100.0,100.0,0.0,100.0,...,24.0,0.0,0.0,99.0,230655.0,66865.0,100.0,7.736001e+11,100.0,0
60056,31353,2019-01-01,100.0,0.0,31865.0,24.0,100.0,100.0,0.0,100.0,...,22.0,0.0,420.0,99.0,139999.0,4536657.0,100.0,8.161921e+11,100.0,0
63196,15777,2019-01-01,100.0,0.0,38825.0,11.0,100.0,100.0,0.0,100.0,...,30.0,0.0,0.0,78.0,6401354.0,17570387.0,100.0,9.882795e+11,100.0,0
67187,28248,2019-01-01,100.0,0.0,31040.0,21.0,100.0,100.0,0.0,100.0,...,22.0,0.0,0.0,99.0,90224.0,4430131.0,100.0,7.949852e+11,100.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171768,100213,2019-06-30,100.0,1.0,43376.0,21.0,99.0,100.0,1.0,100.0,...,25.0,0.0,0.0,87.0,5040257.0,13896589.0,100.0,1.104076e+12,99.0,0
17172181,109655,2019-06-30,100.0,0.0,41572.0,15.0,100.0,100.0,0.0,100.0,...,25.0,0.0,0.0,95.0,2446499.0,2600324.0,100.0,1.056712e+12,100.0,0
17172630,115686,2019-06-30,100.0,0.0,32153.0,20.0,100.0,100.0,0.0,100.0,...,28.0,0.0,0.0,98.0,71363.0,93148.0,100.0,8.205413e+11,100.0,0
17159123,60438,2019-06-30,100.0,0.0,33065.0,23.0,100.0,100.0,0.0,100.0,...,27.0,0.0,0.0,91.0,4268979.0,2072669.0,100.0,8.290822e+11,100.0,0


In [190]:
selecetd_label_0[selecetd_label_0['label']==1]

,disk_id,ds,n_5,r_5,r_9,r_12,n_170,n_171,r_171,n_172,...,r_194,r_197,r_199,n_233,r_241,r_242,n_175,r_175,n_232,label


In [194]:
from tqdm import tqdm

# final_df와 final_df2를 합칩니다.
final_df = pd.concat([selected_label_1, selecetd_label_0])

# model, disk_id, date를 기준으로 그룹화하고 shuffle합니다.
grouped = final_df.groupby(['disk_id'])
shuffled_groups = random.sample(list(grouped), len(grouped))

# 진행 상황을 표시하기 위해 tqdm 사용
shuffled_df = pd.DataFrame()
for _, group in tqdm(shuffled_groups):
    shuffled_df = pd.concat([shuffled_df, group])

# 최종 결과를 출력합니다.
shuffled_df

C:\Users\choib\AppData\Local\Temp\ipykernel_46088\48649565.py:8: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  shuffled_groups = random.sample(list(grouped), len(grouped))
  0%|          | 0/84 [00:00<?, ?it/s]

100%|██████████| 84/84 [00:00<00:00, 429.59it/s]


,disk_id,ds,n_5,r_5,r_9,r_12,n_170,n_171,r_171,n_172,...,r_194,r_197,r_199,n_233,r_241,r_242,n_175,r_175,n_232,label
71241,4476,2019-01-01,100.0,0.0,31975.0,20.0,100.0,100.0,0.0,100.0,...,26.0,0.0,0.0,90.0,4531745.0,2105874.0,100.0,8.122004e+11,100.0,0
178109,4476,2019-01-02,100.0,0.0,31999.0,20.0,100.0,100.0,0.0,100.0,...,26.0,0.0,0.0,90.0,4537193.0,2108737.0,100.0,8.122944e+11,100.0,0
288909,4476,2019-01-03,100.0,0.0,32022.0,20.0,100.0,100.0,0.0,100.0,...,26.0,0.0,0.0,90.0,4542597.0,2111736.0,100.0,8.123883e+11,100.0,0
413765,4476,2019-01-04,100.0,0.0,32046.0,20.0,100.0,100.0,0.0,100.0,...,26.0,0.0,0.0,90.0,4548500.0,2114760.0,100.0,8.161166e+11,100.0,0
447280,4476,2019-01-05,100.0,0.0,32070.0,20.0,100.0,100.0,0.0,100.0,...,26.0,0.0,0.0,90.0,4560684.0,2117913.0,100.0,8.162106e+11,100.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16827535,48636,2019-06-26,100.0,0.0,33125.0,19.0,100.0,100.0,0.0,100.0,...,28.0,0.0,159.0,94.0,1783712.0,4635130.0,100.0,8.376200e+11,100.0,0
16913176,48636,2019-06-27,100.0,0.0,33149.0,19.0,100.0,100.0,0.0,100.0,...,28.0,0.0,159.0,94.0,1783953.0,4635300.0,100.0,8.377139e+11,100.0,0
16983216,48636,2019-06-28,100.0,0.0,33173.0,19.0,100.0,100.0,0.0,100.0,...,28.0,0.0,159.0,94.0,1784197.0,4635482.0,100.0,8.378078e+11,100.0,0
17116099,48636,2019-06-29,100.0,0.0,33197.0,19.0,100.0,100.0,0.0,100.0,...,27.0,0.0,159.0,94.0,1784442.0,4635667.0,100.0,8.379017e+11,100.0,0


In [195]:
shuffled_df['label'].value_counts()

0    10766
1      824
Name: label, dtype: int64

요약하자면 fail된 디스크에서 fail이후에 있는 디스크가 있는 행을 제거하고 fail됐을때의 날짜의 한달전을 <br>
기준으로 label을 1로 만들고 fail된 디스크만큼 healthy한 디스크를 가져와서 병합하는 작업을 거쳤다.<br>
그리고 그렇게 병합한 dataframe을 final_data_MA1_after_deleted.csv에 저장한다. 

In [192]:
shuffled_df.to_csv('../ssd_data/final_data/final_data_MA2_after_deleted.csv',index=False)

In [196]:
shuffled_df.to_csv('../ssd_data/final_data/final_data_MA2.csv',index=False)